In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate.servers import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables
from will_it_saturate.clients import (
    HttpxClient,
    AioHttpClient,
    WrkClient,
    BaseClient,
)

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
# dont_test

# create server control client + server (needed for turning files into urls)
server_control_host = Host(name="macmini.fritz.box", port=8001)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server = server_control_client.get_or_create_server(FastAPIUvicornServer(host="macmini.fritz.box", port=5001))
print(server)


protocol='http' name='fastAPI/uvicorn' host='localhost' port=5001


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

# create server control client + server (needed for turning files into urls)
server_control_host = Host(name="macmini.fritz.box", port=8001)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server = server_control_client.get_or_create_server(FastAPIUvicornServer(host="macmini."))
print(server)

# create files and urls
byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)
files = server_control_client.get_or_create_files(epoch)
print(files[:10])
epoch.urls = [server.file_to_url(file) for file in files]

# create control client + measure server
client_control_host = Host(name="localhost", port=8001)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_client = HttpxClient(name="httpx")

result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

protocol='http' name='fastAPI/uvicorn' host='localhost' port=5001
[BenchmarkFile(number=0, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='d41d8cd98f00b204e9800998ecf8427e', creator_name='filesystem'), BenchmarkFile(number=1, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='c2b87744cfbdf5a1bb0c3b691ca40e66', creator_name='filesystem'), BenchmarkFile(number=2, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='b197317f87369090fe0ae4bbfe90ddf3', creator_name='filesystem'), BenchmarkFile(number=3, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='0830427b0a6a8e5831fafa2babef13c1', creator_name='filesystem'), BenchmarkFile(number=4, base_path='10000000_3_125000000', size=10000000, data_root='data', hostname='localhost', port=8000, checksum='46d44b3d9b0

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.elapsed = client_control_client.measure(benchmark_client, epoch)
print(result.elapsed)

# result.elapsed = benchmark_client.measure(epoch)

3.159423792


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.bytes_per_second

118850051.28710909

<IPython.core.display.Javascript object>

# Save Result

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("results.db")
register_default_tables(repository)
results = repository.tables["result"]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result_id = results.add_result(result)
print(result_id)

1


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results.get_results()

[Result(server=FastAPIUvicornServer(protocol='http', name='fastAPI/uvicorn', host='localhost', port=5001), client=HttpxClient(name='httpx'), server_details=HostDetails(machine_id='C07XJ1E5JYVX', cpuinfo={'python_version': '3.9.4.final.0 (64 bit)', 'cpuinfo_version': [8, 0, 0], 'cpuinfo_version_string': '8.0.0', 'arch': 'X86_64', 'bits': 64, 'count': 6, 'arch_string_raw': 'x86_64', 'vendor_id_raw': 'GenuineIntel', 'brand_raw': 'Intel(R) Core(TM) i5-8500B CPU @ 3.00GHz', 'hz_advertised_friendly': '3.0000 GHz', 'hz_actual_friendly': '3.0000 GHz', 'hz_advertised': [3000000000, 0], 'hz_actual': [3000000000, 0], 'l2_cache_size': 262144, 'stepping': 10, 'model': 158, 'family': 6, 'flags': ['1gbpage', '3dnowprefetch', 'abm', 'acpi', 'adx', 'aes', 'apic', 'avx', 'avx1.0', 'avx2', 'bmi1', 'bmi2', 'clflush', 'clflushopt', 'clfsh', 'clfsopt', 'cmov', 'cx16', 'cx8', 'de', 'ds', 'ds_cpl', 'dscpl', 'dtes64', 'dts', 'em64t', 'erms', 'est', 'f16c', 'fma', 'fpu', 'fpu_csds', 'fxsr', 'hle', 'ht', 'htt', 

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

pd.DataFrame([r.dict_for_pandas() for r in results.get_results()])

,server,client,server_host,client_host,elapsed,file_size,file_size_h,complete_size,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,C07XJ1E5JYVX,C02DR0MZQ6LT,3.139357,10000000,9.54MB,375000000,1.194512e+08,113.92MB


<IPython.core.display.Javascript object>